In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-rx1osk3x
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-rx1osk3x
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12773 sha256=11270087cfc6b4f4d58d935f008c87c3f39000c579ad56c5cae879b3b0970041
  Stored in directory: /tmp/pip-ephem-wheel-cache-linmjgwp/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
# 새로운 데이터셋 불러오는 코드
import pandas as pd
dataset_train1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/nlp/dataset.csv')
dataset_train1.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [ ]:
# 감정 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(dataset_train1['Emotion'])
dataset_train1['Emotion'] = encoder.transform(dataset_train1['Emotion'])
dataset_train1.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,0
1,그냥 내 느낌일뿐겠지?,0
2,아직너무초기라서 그런거죠?,0
3,유치원버스 사고 낫다던데,0
4,근데 원래이런거맞나요,0


In [ ]:
# 라벨링된 감정 매핑 {0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복', 6: '혐오'}
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복', 6: '혐오'}

In [ ]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset_train1, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 30875
test shape is: 7719


In [ ]:
print(train.head())
print(test.head())

                                                Sentence  Emotion
29089                         오오~~바르샤가 좀 더 우승하겠는데~!!ㅋㅋㅋㅋ        5
24337  조연도 안되는급인데 이렇게 언플하고도 조연드립치고 쉴드하는 개셈 보면 왜 쓰레기라하...        4
20228                 남친 유격훈련가서 연락이 안되여ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ        3
18840                               어제전화왔는데 많이힘들어하던데 ...        3
10369                             아님 그냥 그대로 남자소유가 되는거임??        1
                       Sentence  Emotion
34278                   어디서개구라를        6
11355           설마 진짜 믿는건 아니겠지?        1
26580     끝까지 읽었다고는 말 안했다...ㅎㅎㅎ        4
217           교도소에서 골목 대장이 되버렸네        0
23737  돈만 주면 맛집되는 참 엉터리 맛집선정...        4


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
# 테스트데이터셋을 이용한 테스트방식을 위한 데이터변환기
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset['Sentence']]
        self.labels = [np.int32(i) for i in dataset['Emotion']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 실시간 입력 형식을 위한 데이터변환기
class BERTDataset2(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
data_train = BERTDataset(train,  tok, max_len, True, False)
data_test = BERTDataset(test, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# AdamW
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# SGD
#optimizer = optim.SGD(optimizer_grouped_parameters, lr=0.01, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# train
'''
PATH = '/content/drive/MyDrive/Colab Notebooks/nlp/weights/'
 
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train loss {} train acc {}".format(e+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        test_acc += calc_accuracy(out, label)
    print("epoch {} test loss {} test acc {}".format(e+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))
'''
  
'''
    # epoch당 한번씩 model을 저장할 때 사용
    torch.save({
        'epoch' : e+1,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict' : optimizer.state_dict()
    }, PATH + 'model_{}.tar'.format(e+1))
'''

'''
# 마지막에 한번만 model을 저장할 때 사용
torch.save({
    'epoch' : 9,
    'model_state_dict' : model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict()
}, PATH + 'model_{}_2.tar'.format(e+1))
'''

"\ntorch.save({\n    'epoch' : 9,\n    'model_state_dict' : model.state_dict(),\n    'optimizer_state_dict' : optimizer.state_dict()\n}, PATH + 'model_{}_2.tar'.format(e+1))\n"

In [ ]:
#모델 초기화
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#모델 state_dict 불러오기
model1.eval()
PATH = '/content/drive/MyDrive/Colab Notebooks/nlp/weights/'
checkpoint = torch.load(PATH + 'model_10_1.tar')
model1.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
# 테스트파일로 테스트하기

# 테스트파일 불러오기
dataset_test1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/nlp/test.csv')

test_set = BERTDataset(dataset_test1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

newtest_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model1(token_ids, valid_length, segment_ids)
  newtest_acc += calc_accuracy(out, label)
  print(out)
  print(out.argmax(), label, "\n")
print("test acc {}" .format(newtest_acc /(batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


tensor([[ 4.0118, -0.1726, -2.5907,  6.3703, -2.1629, -1.0503, -2.9885]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor(3, device='cuda:0') tensor([0], device='cuda:0') 

tensor([[-1.7977, -1.3352, -1.6244, -1.6279, -1.0812,  8.4428, -0.5522]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor(5, device='cuda:0') tensor([5], device='cuda:0') 

tensor([[ 4.7621,  4.7689, -1.7507, -1.5604, -2.0395, -2.8497, -1.5863]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor(1, device='cuda:0') tensor([6], device='cuda:0') 

tensor([[ 0.7515, -0.2049, -1.2873, -0.5770, -1.8498, -2.3053,  5.6463]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor(6, device='cuda:0') tensor([6], device='cuda:0') 

tensor([[-0.2845, -1.1097, -1.6651,  7.6573, -1.7484, -0.4590, -1.4098]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor(3, device='cuda:0') tensor([3], device='cuda:0') 

tensor([[-1.4167, -1.4797,  6.9720, -0.9932, -0.9105, -2.1389,  0.0767]],
       device='cuda:0

In [ ]:
# 해당 인덱스값이 어떤 감정을 나타내는지 바꿔줌.
def switch_feel(feel_num):
    if feel_num == 0:
      return "공포"
    elif feel_num == 1:
      return "놀람"
    elif feel_num == 2:
      return "분노"
    elif feel_num == 3:
      return "슬픔"
    elif feel_num == 4:
      return "중립"
    elif feel_num == 5:
      return "행복"
    elif feel_num == 6:
      return "혐오"
    return "무감정"

# 실시간 입력형 테스트방식
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset2(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            best = np.argmax(logits)
            test_eval.append(best)
            #가장 높은것의 값을 -10으로 바꾸어 그 다음으로 높은값을 argmax로 쉽게 찾을 수 있게 함.
            logits[best] = -10
            test_eval.append(np.argmax(logits))
            
        # 두번째로 높은 값이 0보다 작을경우에는 입력값의 감정과 연관이 없다고 판단되어 결과출력에서 제외함.    
        if logits[test_eval[1]] < 0:
            print(">> 입력내용에서 가장 크게 느껴지는 감정은 " + switch_feel(test_eval[0]) + " 입니다.")
        else:
            print(">> 입력내용에서 가장 크게 느껴지는 감정은 " + switch_feel(test_eval[0]) + " 이고, 두번째는 " + switch_feel(test_eval[1]) + " 입니다.")

In [ ]:
#입출력 반복 실행. 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("입력(0 입력시 종료) : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

입력 : 아파요


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[ 0.28464088 -0.9840919  -1.6508105   7.66206    -1.7783087  -0.77620155
 -1.7528187 ]
[  0.28464088  -0.9840919   -1.6508105  -10.          -1.7783087
  -0.77620155  -1.7528187 ]
>> 입력내용에서 가장 크게 느껴지는 감정은 슬픔 이고, 두번째는 공포 입니다.


입력 : ㅋㅋㅋ개웃겨
[-1.730282   -1.4707247  -1.6992302  -1.5970664  -0.8935918   8.438041
 -0.57226175]
[ -1.730282    -1.4707247   -1.6992302   -1.5970664   -0.8935918
 -10.          -0.57226175]
>> 입력내용에서 가장 크게 느껴지는 감정은 행복 입니다.


입력 : 0
